# Run Lifts

In [31]:
import time
import pandas as pd
from tkinter import Tk, Canvas
import numpy as np
from datetime import datetime, timedelta
import random

## Set Simulation Parameters

In [32]:
# Parameters
num_entries = 30  # Number of people
floors = 6  # Number of floors in the building
start_date = datetime(2024, 8, 1, 8, 0)  # Start time
end_date = datetime(2024, 8, 1, 8, 4)  # End time

## Create entries DataFrame

In [33]:
def random_date(start, end):
    """Generate a random datetime between `start` and `end`."""
    delta = end - start
    random_seconds = random.randint(0, int(delta.total_seconds()))
    return start + timedelta(seconds=random_seconds)

def biased_floor_selection(floors, bias_floor, bias_probability):
    """Select a floor with a given bias probability for the bias_floor."""
    if random.random() < bias_probability:
        return bias_floor
    else:
        return random.randint(2, floors)  # Avoid selecting floor 1 too often randomly

data = []

for _ in range(num_entries):
    timestamp = random_date(start_date, end_date)
    
    if timestamp.hour < 12:  # Morning entries biased towards floor 1
        entry_floor = biased_floor_selection(floors - 1, 1, 0.7)
    else:  # Afternoon/evening entries less biased
        entry_floor = biased_floor_selection(floors - 1, 1, 0.2)
    
    if timestamp.hour >= 16:  # Evening exits biased towards floor 1
        exit_floor = biased_floor_selection(floors - 1, 1, 0.7)
    else:  # Morning/afternoon exits less biased
        exit_floor = biased_floor_selection(floors - 1, 1, 0.2)
    
    # Ensure entry_floor and exit_floor are not the same
    while entry_floor == exit_floor:
        exit_floor = random.randint(1, floors - 1)
    
    data.append([timestamp, entry_floor, exit_floor])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Timestamp', 'Entry_Floor', 'Exit_Floor']).sort_values('Timestamp')

In [34]:
df.head()

,Timestamp,Entry_Floor,Exit_Floor
6,2024-08-01 08:00:18,1,2
0,2024-08-01 08:00:23,1,4
19,2024-08-01 08:00:27,1,3
14,2024-08-01 08:00:27,1,3
18,2024-08-01 08:00:28,1,2


## Example Functions

In [41]:
def efficient_algorithm(elevator_population, floor_population, floors, elevator_floor, t_floor):
    """If the elevator has passengers, prioritize their destinations first"""
    if elevator_population:
        # Find the nearest destination for current passengers
        closest_floor = min(elevator_population, key=lambda x: abs(x - elevator_floor))

    # If there are no passengers, consider picking up people from other floors
    elif any(floor_population):  # Check if there's anyone waiting on any floor
        # Sort floors by proximity to the current elevator position
        closest_floor = min((floor for floor in range(floors) if floor_population[floor] > 0), 
                            key=lambda x: abs(x - elevator_floor))
    else:
        closest_floor = elevator_floor  # No one in the elevator and no one waiting

    return closest_floor

def baseline_algorithm(elev_pop, floor_population, floors, elevator_floor, t_floor):
    """Simple logic: If the lift is at the bottom floor, head upwards. If the lift is at the 
    top floor, head back down. If in the middle, use the previous direction"""
    if elevator_floor == 1:
        return floors
    elif elevator_floor == floors:
        return 1

    return t_floor

## Simulation Code

In [42]:
class Person:
    population = 0

    def __init__(self, entry_time, start_floor, target_floor):
        self.animation = None
        self.id = Person.population
        Person.population += 1
        self.entry_time = entry_time
        self.start_floor = start_floor
        self.target_floor = target_floor
        self.direction = 1 if self.start_floor < self.target_floor else -1
        self.finished = False
        self.in_elevator = False
        self.wait_time = 0
        self.elevator_spot = False

    def arrived(self, floor):
        return self.target_floor == floor

    def waiting(self):
        return not self.in_elevator and not self.finished

def single_simulation(algorithm, data, floors, max_elevator_capacity=6, animate=True, animation_speed=1):
    start_lifts = time.perf_counter()

    assert algorithm in ["baseline", "efficient"], "That algorithm is not supported."
    
    if len(data) < 2 or floors < 2:
        return 0
    
    floor_height = round(600 / floors)
    total_population = []
    elevator_population = []
    floor_population = [0] * floors
    elevator_floor = 1
    target_floor = 1
    t_floor = 1
    elevator_direction = 0.5
    elev_pop = []
    start_time = data['Timestamp'].min()
    end_time = data['Timestamp'].max()
    current_time = start_time

    if animate:
        arrivals_population = [0] * floors
        elevator_animation = [0] * max_elevator_capacity
        tk = Tk()
        tk.attributes("-fullscreen", False)
        canvas = Canvas(tk, width=1000, height=1000)
        tk.title(f'Elevator - {algorithm} algorithm')
        canvas.pack()

        def move_slowly(animation, x, y):
            total_move_time = 0.15  # Total time to move the animation in seconds
            steps = 50  # Number of steps to divide the move into
            for j in range(steps):
                canvas.move(animation, x / steps, y / steps)
                tk.update()
                time.sleep(total_move_time / steps)
            tk.after(10000, canvas.delete, animation)  # Disappear after 10 seconds

        canvas.create_oval(85, 70, 95, 80, fill='black')
        canvas.create_oval(85, 90, 95, 100, fill='white')
        canvas.create_oval(85, 110, 95, 120, fill='green')
        waiting_label = canvas.create_text(100, 75, text='Waiting', anchor='w')
        inside_label = canvas.create_text(100, 95, text='Inside elevator', anchor='w')
        delivered_label = canvas.create_text(100, 115, text='Arrived', anchor='w')
        
        clock_label = canvas.create_text(100, 135, text='Time: ' + current_time.strftime('%H:%M:%S'), anchor='w')

        for k in range(1, floors+1):
            canvas.create_line(50, 200 + (floors - k) * floor_height, 600, 200 + (floors - k) * floor_height)
            if k <= floors - 1:
                canvas.create_text(5, 200 + (floors - k) * floor_height, text='Floor ' + str(k), anchor='w')
        
        canvas.create_rectangle(200, 200, 400, 200 + floor_height * (floors-1))
        elevator = canvas.create_rectangle(203, 200 + floor_height * (floors - 1 - elevator_floor), 397, 200 + floor_height * (floors - elevator_floor), fill='black')
        tk.update()

    while any(not person.finished for person in total_population) or current_time <= end_time:
        new_people = data[data['Timestamp'] == current_time]

        for _, row in new_people.iterrows():
            timestamp = row['Timestamp']
            start_floor = row['Entry_Floor']
            target_floor = row['Exit_Floor']
            
            if start_floor < 0 or start_floor >= floors or target_floor < 0 or target_floor >= floors:
                print(f"Invalid floor values: start_floor={start_floor}, target_floor={target_floor}")
                continue
            person = Person(current_time, start_floor, target_floor)
            s_f = person.start_floor
            
            if animate:
                offset = floor_population[s_f] * 13
                person.animation = canvas.create_oval(185 - offset, 190 + (floors - s_f) * floor_height, 195 - offset, 200 + (floors - s_f) * floor_height, fill='black')
                tk.update()
            
            floor_population[s_f] += 1
            total_population.append(person)

        for person in total_population:
            person.wait_time += 1 if not person.finished else 0

            if person.in_elevator and person.arrived(elevator_floor):
                person.in_elevator = False
                person.finished = True
                elevator_population.remove(person)
                elevator_buttons[person.target_floor] = False
                elev_pop = [i for i in range(len(elevator_buttons)) if elevator_buttons[i]]

                if animate:
                    elevator_animation[person.elevator_spot] = False
                    canvas.itemconfig(person.animation, fill='green')
                    elevator_floor = int(elevator_floor)
                    arrivals_population[elevator_floor] += 1
                    move_slowly(person.animation, 390 + arrivals_population[elevator_floor] * 12 - canvas.coords(person.animation)[0], 15 * (person.elevator_spot % 2))
                    canvas.itemconfig(delivered_label, text='Arrived - ' + str((len(total_population) - sum(floor_population) - len(elevator_population))))
                    canvas.itemconfig(inside_label, text='Inside elevator - ' + str(len(elevator_population)))
                    canvas.itemconfig(waiting_label, text='Waiting - ' + str(sum(floor_population)))

        if algorithm == "efficient": 
            t_floor = efficient_algorithm(elev_pop, floor_population, floors, elevator_floor, t_floor)
            if t_floor > elevator_floor: elevator_direction = 0.5
            elif t_floor < elevator_floor: elevator_direction = -0.5
            else: elevator_direction = 0
            #elevator_direction = elevator_direction * 0.5
        else:
            t_floor = baseline_algorithm(elev_pop, floor_population, floors, elevator_floor, t_floor)
            if t_floor > elevator_floor: elevator_direction = 0.5
            elif t_floor < elevator_floor: elevator_direction = -0.5
            

        for person in reversed(total_population):
            if person.waiting() and person.start_floor == elevator_floor and len(elevator_population) < max_elevator_capacity:

                elevator_population.append(person)
                person.in_elevator = True
                floor_population[int(elevator_floor)] -= 1
                elevator_buttons = [False] * floors
                for person in elevator_population:
                    elevator_buttons[person.target_floor] = True
                elev_pop = [i for i in range(len(elevator_buttons)) if elevator_buttons[i]]
                
                if animate:
                    for spot in range(len(elevator_animation)):
                        if not elevator_animation[spot]:
                            elevator_animation[spot] = True
                            person.elevator_spot = spot
                            move_slowly(person.animation, (275 + (spot % 3) * 15) - canvas.coords(person.animation)[0], -15 * (spot % 2))
                            break
                    canvas.itemconfig(person.animation, fill='white')
                    canvas.itemconfig(inside_label, text='Inside elevator - ' + str(len(elevator_population)))
                    canvas.itemconfig(waiting_label, text='Waiting - ' + str(sum(floor_population)))


        if animate:
            current_time += timedelta(seconds=1)
            canvas.itemconfig(clock_label, text='Time: ' + current_time.strftime('%H:%M:%S'))
            for i in range(floor_height):
                tk.update()
                time.sleep(animation_speed / floor_height)
                canvas.move(elevator, 0, -elevator_direction)
                for person in elevator_population:
                    canvas.move(person.animation, 0, -elevator_direction)
        else:
            current_time += timedelta(seconds=1)
    
        elevator_floor += elevator_direction

    wait_times = [person.wait_time for person in total_population if person.finished]
    average_wait_time = sum(wait_times) / len(wait_times) if len(wait_times) > 0 else 1
    # Convert the timestamp string to a datetime object
    # Define 8:00 AM on the same day
    eight_am = datetime(current_time.year, current_time.month, current_time.day, 8, 0, 0)
    
    # Calculate the difference
    time_difference = current_time - eight_am
    
    # Extract hours, minutes, and seconds from the time difference
    hours, remainder = divmod(time_difference.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    if animate:
        canvas.create_text(200, 900, text=f"Total time elapsed: {hours} hours, {minutes} minutes, and {seconds} seconds", font=("Cambria", 12))
        canvas.create_text(200, 915, text=f'Shortest wait time: {min(wait_times)}s', font=("Cambria", 12))
        canvas.create_text(200, 930, text=f'Longest wait time: {max(wait_times)}s', font=("Cambria", 12))
        canvas.create_text(200, 945, text="Average wait time: " + str(round(average_wait_time, 2)), font=("Cambria", 12))
        tk.mainloop()
        
    # Print the results
    print(f"Total time elapsed: {hours} hours, {minutes} minutes, and {seconds} seconds")
    print(f'Shortest wait time: {min(wait_times)}s')
    print(f'Longest wait time: {max(wait_times)}s')
    print("Average wait time: " + str(round(average_wait_time, 1)))    


## Run simulations

In [43]:
# With animations
single_simulation(algorithm="efficient", data=df, floors=floors, max_elevator_capacity=6, animate=True, animation_speed=0.1)

TclError: invalid command name ".!canvas"

In [38]:
# Run the efficient simulation with animations off
single_simulation(algorithm="efficient", data=df, floors=floors, max_elevator_capacity=6, animate=False, animation_speed=0.1)

Total time elapsed: 0 hours, 3 minutes, and 51 seconds
Shortest wait time: 4s
Longest wait time: 28s
Average wait time: 11.9


In [39]:
# Run the baseline simulation with animations off
single_simulation(algorithm="baseline", data=df, floors=floors, max_elevator_capacity=6, animate=False, animation_speed=0.1)


Total time elapsed: 0 hours, 4 minutes, and 1 seconds
Shortest wait time: 3s
Longest wait time: 41s
Average wait time: 14.5
